In [1]:
import pandas as pd

In [2]:
jan_data = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [3]:
#Q1
# How many columns are in the dataframe?
len(jan_data.columns)

19

In [4]:
# Display the first few rows of the DataFrame
jan_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
# Q2
# Computing the duration of each trip
jan_data['duration'] = jan_data.tpep_dropoff_datetime - jan_data.tpep_pickup_datetime
jan_data['duration'] = jan_data['duration'].dt.total_seconds() / 60  # Convert to minutes

In [6]:
# Standard deviation of the duration
std_duration = jan_data['duration'].std()
# Display the standard deviation
print(f"Standard deviation of trip duration: {std_duration:.2f} minutes")

Standard deviation of trip duration: 42.59 minutes


In [7]:
# Q3
# keep the records with a duration between 1 and 60 minutes
df = jan_data[(jan_data['duration'] >= 1) & (jan_data['duration'] <= 60)]
# What fraction of the records were kept?
fraction_kept = len(df) / len(jan_data)
# Display the fraction
print(f"Fraction of records kept: {fraction_kept:.2%}")

Fraction of records kept: 98.12%


In [8]:
# Q4
# One hot encoding to the pickup and dropoff location IDs
# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings)
# Fit a dictionary vectorizer to the pickup and dropoff location IDs
# Get a feature matrix
from sklearn.feature_extraction import DictVectorizer
import numpy as np

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer(dtype=np.int8)
X = dv.fit_transform(train_dicts)
# Dimensionality of the feature matrix
n_samples, n_features = X.shape
# Display the dimensionality
print(f"Dimensionality of the feature matrix: {n_samples} samples, {n_features} features")


/tmp/ipykernel_4064/3318945960.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


Dimensionality of the feature matrix: 3009173 samples, 515 features


In [ ]:
# Q5
# Training a linear regression model with default parameters, where duration is the target
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error


# Train the model on the whole dataset
y = df['duration'].values


model = LinearRegression()
model.fit(X, y)
# Predict on the validation set
y_pred = model.predict(X)


In [11]:
# Calculate the RMSE
rmse = root_mean_squared_error(y, y_pred)
# Display the RMSE
print(f"RMSE: {rmse:.2f} minutes")

RMSE: 7.65 minutes


In [13]:
# Q6: Evaluate the model on a validation set
# Let's use February data for validation
feb_data = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
# Compute the duration of each trip
feb_data['duration'] = feb_data.tpep_dropoff_datetime - feb_data.tpep_pickup_datetime
feb_data['duration'] = feb_data['duration'].dt.total_seconds() / 60  # Convert to minutes
# Keep the records with a duration between 1 and 60 minutes
feb_data = feb_data[(feb_data['duration'] >= 1) & (feb_data['duration'] <= 60)]
# One hot encoding to the pickup and dropoff location IDs
feb_data[categorical] = feb_data[categorical].astype(str)
# Turn the dataframe into a list of dictionaries
val_dicts = feb_data[categorical].to_dict(orient='records')
# Get a feature matrix
X_val = dv.transform(val_dicts)
# Predict on the validation set
y_val = feb_data['duration'].values
y_val_pred = model.predict(X_val)
# Calculate the RMSE
rmse_val = root_mean_squared_error(y_val, y_val_pred)
# Display the RMSE
print(f"Validation RMSE: {rmse_val:.2f} minutes")


Validation RMSE: 7.81 minutes


In [14]:
# Q7: Save the model
import pickle
# Save the model to a file
with open('model.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out) 